In [ ]:

import numpy as np
import h5py
import open3d as o3d
from pathlib import Path
import trimesh
import smplx

In [ ]:
h5file = "demo/demo_lidar/Hong Lidar 13.hdf5"
save_dir = Path(h5file).parent / "frames"
save_dir.mkdir(parents=True, exist_ok=True)
min_amplitude = 0.0

In [496]:
smplmodel_folder = "smpl_models"
gender = "male"  # male/female/neutral

smplmodel = smplx.create(
    smplmodel_folder, model_type="smpl", gender=gender, ext="pkl"
)
faces = smplmodel.faces

In [574]:
def create_cloud_from_distance_image(
    distance: np.array, rays: np.array, horizontal_binning: bool, vertical_binning: bool
):
    # map the distance image (m x n) to a point cloud (m x n x 3)
    slicing = [1, 1]
    if horizontal_binning:
        slicing[0] = 2
    if vertical_binning:
        slicing[1] = 2
    # take subset of rays based on binning configuration
    rays_small = rays[:: slicing[1], :: slicing[0], :]
    cloud = distance[..., np.newaxis] * rays_small
    return cloud


h5 = h5py.File(h5file, "r")
rays = h5["recording_information"]["rays"][()]

# for key in h5["frames"].keys():
#     idx = int(key)
#     if idx < 0:
#         continue

idx = 0

key = f"{idx:08d}"
print("showing frame: ", key)
distance = h5["frames"][key]["Distance"][()] / 1000
amplitude = h5["frames"][key]["Amplitude"][()]

cloud = create_cloud_from_distance_image(distance, rays, False, False)
keep = (
    (amplitude > 100)
    # & (distance < 4.4)
    & (distance < 4.7)
    & (distance > 3)
    # & (cloud[..., 0] < 4.3)
    # & (cloud[..., 0] > 3)
    # & (cloud[..., 1] > -1.5)
    # & (cloud[..., 1] < 1.5)
    # & (cloud[..., 2] > -1.9)
    # & (cloud[..., 2] < 1.8)
)
keep[:, -160:] &=  (distance[:, -160:] < 4.4)

border = 70
keep[:, 0:border] = False
keep[:, -border:] = False

cloud = cloud[keep]
amplitude = amplitude[keep]

print(cloud[:, 2].max(), cloud[:, 2].min())


# normalize the height to [-1, 1]
scale = (cloud[:, 2].max() - cloud[:, 2].min()) * 0.5
cloud /= scale
cloud[:, 2] = (cloud[:, 2] - cloud[:, 2].min()) - 1

# shift depth to 0
cloud[:, 0] -= cloud[:, 0].min()

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(cloud[:, [0, 2, 1]])

o3d.io.write_point_cloud(str(save_dir / f"{key}.ply"), pcd, write_ascii=False)

# o3d.visualization.draw_geometries([pcd], )

showing frame:  00000000
1.398978270821253 -1.9875291604358045


True

In [582]:
distance.shape

(240, 320)

In [576]:
pcd:o3d.geometry.PointCloud = o3d.io.read_point_cloud(f"demo/demo_lidar/frames/{key}.ply")
pcd_example: o3d.geometry.PointCloud = o3d.io.read_point_cloud("demo/demo_depth/00003200.ply")

mesh_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(
    size=0.6, origin=[-1, -1, 0]
)
o3d.visualization.draw_geometries([pcd, mesh_frame, ])



In [577]:
idx = 0
key = f"{idx:08d}"
pcd_em = o3d.io.read_point_cloud(f"demo/demo_lidar/output/{key}_EM.ply")

pcd2 = o3d.io.read_point_cloud(f"demo/demo_lidar/output/{key}.ply")
points = np.asarray(pcd2.points)
points[:, 2] += 1
pcd2.points = o3d.utility.Vector3dVector(points)

o3d.visualization.draw_geometries([pcd_em, mesh_frame, pcd2])

In [ ]:
from src.extract_measurements import extract_measurements

In [578]:
import torch
import pprint

idx = 0
key = f"{idx:08d}"
smpl_data = torch.load(f"demo/demo_lidar/output/{key}_EM.pth", map_location="cpu")
pprint.pprint({k: v.shape if isinstance(v, torch.Tensor) else v for k, v in smpl_data.items()})
vertices = smpl_data["vertices"].data.cpu().numpy()[0]

extract_measurements(height=170, vertices=vertices)

mesh = trimesh.Trimesh(vertices=vertices, faces=smplmodel.faces, process=False)
mesh.show()

{'betas': torch.Size([1, 10]),
 'body_pose': torch.Size([1, 69]),
 'full_pose': None,
 'global_orient': torch.Size([1, 3]),
 'joints': torch.Size([1, 45, 3]),
 'transl': None,
 'v_shaped': None,
 'vertices': torch.Size([1, 6890, 3])}
height: 170.000000
waist: 91.867178
belly: 89.964772
chest: 99.123903
wrist: 16.606401
neck: 36.017457
arm length: 52.563364
thigh: 54.131230
shoulder width: 44.948353
hips: 100.751455
ankle: 21.265298


In [579]:
# To A pose
body_pose = torch.zeros_like(smpl_data["body_pose"])
body_pose[:, 50-3] = 5.6
body_pose[:, 53-3] = -5.6

output = smplmodel(betas=smpl_data["betas"], body_pose=body_pose)
vertices_a = output.vertices.data.cpu().numpy()[0]

In [580]:
extract_measurements(height=170, vertices=vertices_a)

height: 170.000000
waist: 92.744790
belly: 91.496706
chest: 102.601306
wrist: 16.887352
neck: 37.078121
arm length: 55.279225
thigh: 54.449998
shoulder width: 44.883795
hips: 100.415738
ankle: 21.217909


In [573]:
mesh = trimesh.Trimesh(vertices=vertices_a, faces=smplmodel.faces, process=False)
# o3d.visualization.draw_geometries([mesh.as_open3d])
mesh.show()

In [566]:
from body_measurements.measurement import Body3D


body = Body3D(vertices_a, faces)

body_measurements = body.getMeasurements()

height = body.height()
weight = body.weight()
shoulder_2d, shoulder_location, shoulder_length = body.shoulder()
chest_2d, chest_location, chest_length = body.chest()
hip_2d, hip_location, hip_length = body.hip()
waist_2d, waist_location, waist_length = body.waist()
thigh_2d, thigh_location, thigh_length = body.thighOutline()
outer_leg_length = body.outerLeg()
inner_leg_length = body.innerLeg()
neck_2d, neck_location, neck_length = body.neck()
neck_hip_length = body.neckToHip()

In [567]:
names = "height, shoulder_length, chest_length, hip_length, waist_length, thigh_length, outer_leg_length, inner_leg_length, neck_length, neck_hip_length".split(", ")
pprint.pprint({k: v for k, v in zip(names, body_measurements[1:])})

{'chest_length': 1.0832751063794461,
 'height': 1.9136700564560123,
 'hip_length': 1.08189833432511,
 'inner_leg_length': 0.785,
 'neck_hip_length': 0.65,
 'neck_length': 0.4559352823331366,
 'outer_leg_length': 0.905,
 'shoulder_length': 1.020576747832463,
 'thigh_length': 0.6729787545082321,
 'waist_length': 0.9894477749666172}


In [568]:
height = 170
scale = height / body.height()
body_measurements = np.asarray(body.getMeasurements()[1:]) * scale

In [569]:
names = "height, shoulder_length, chest_length, hip_length, waist_length, thigh_length, outer_leg_length, inner_leg_length, neck_length, neck_hip_length".split(", ")
pprint.pprint({k: v for k, v in zip(names, body_measurements)})

{'chest_length': 96.23224623452161,
 'height': 170.0,
 'hip_length': 96.10994132179775,
 'inner_leg_length': 69.735114237582,
 'neck_hip_length': 57.7424512795265,
 'neck_length': 40.50280127190508,
 'outer_leg_length': 80.3952590891869,
 'shoulder_length': 90.66246636728245,
 'thigh_length': 59.783758375920016,
 'waist_length': 87.89713836868583}
